*Copyright (c) Cornac Authors. All rights reserved.*

*Licensed under the Apache 2.0 License.*

# Model Ensembling

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/PreferredAI/cornac/blob/master/tutorials/model_ensembling.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/PreferredAI/cornac/blob/master/tutorials/model_ensembling.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

This notebook provides an example of how to ensemble multiple recommendation models in Cornac.

Ensemble models is a technique that combines the predictions of multiple models to produce a single prediction. The idea is that by combining the predictions of multiple models, we can improve the overall performance of the recommendation system.

We will use the MovieLens 100K dataset and ensemble 2 models.

** Note: ** This notebook requires the `scikit-learn` package. 

## 1. Setup

### Install required dependencies

In [1]:
! pip install seaborn scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 12.1 MB/s eta 0:00:00


In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

from cornac.datasets import movielens
from cornac.data import Dataset
from cornac.models import BPR, WMF
from cornac.eval_methods import RatioSplit
from cornac.metrics import Precision, Recall
from cornac.utils import cache
from cornac import Experiment

from sklearn.ensemble import BaggingRegressor, RandomForestRegressor, AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor

/Users/darrylong/miniconda3/envs/gcmc-pytorch/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 2. Prepare Experiment

### Loading Dataset

First, we load the MovieLens 100K dataset.

In [2]:
data = movielens.load_feedback(variant="100K") # load dataset
# dataset = Dataset.from_uir(data) # convert to Dataset object

rs = RatioSplit(data, test_size=0.2, seed=42, verbose=True)
train_set, test_set = rs.train_set, rs.test_set

rating_threshold = 1.0
exclude_unknowns = True
---
Training data:
Number of users = 943
Number of items = 1651
Number of ratings = 80000
Max rating = 5.0
Min rating = 1.0
Global mean = 3.5
---
Test data:
Number of users = 943
Number of items = 1651
Number of ratings = 19964
Number of unknown users = 0
Number of unknown items = 0
---
Total users = 943
Total items = 1651


### Training BPR and WMF models

We will train two models: 

1. BPR (Bayesian Personalized Ranking)
2. WMF (Weighted Matrix Factorization)

In [8]:
# Train BPR model
bpr_model = BPR(k=10, max_iter=100, learning_rate=0.01, lambda_reg=0.001)
# Train WMF model
wmf_model = WMF(k=10, max_iter=300, a=1.0, b=0.1, learning_rate=0.001, lambda_u=0.01, lambda_v=0.01,)

models = [bpr_model, wmf_model]
metrics = [Precision(k=10), Recall(k=10)]

experiment = Experiment(rs, models, metrics, user_based=True).run()


[BPR] Training started!

[BPR] Evaluation started!


Ranking: 100%|██████████| 940/940 [00:00<00:00, 1395.84it/s]



[WMF] Training started!


100%|██████████| 300/300 [00:16<00:00, 18.55it/s, loss=172]


Learning completed!

[WMF] Evaluation started!


Ranking: 100%|██████████| 940/940 [00:00<00:00, 2174.64it/s]


TEST:
...
    | Precision@10 | Recall@10 | Train (s) | Test (s)
--- + ------------ + --------- + --------- + --------
BPR |       0.3189 |    0.2015 |    1.2022 |   0.6773
WMF |       0.3135 |    0.2022 |   16.3237 |   0.4366



In [54]:
# X_train, y_train = list(zip(train_set.uir_tuple[0], train_set.uir_tuple[1])), train_set.uir_tuple[2]
# X_test, y_test = list(zip(test_set.uir_tuple[0], test_set.uir_tuple[1])), test_set.uir_tuple[2]

# bpr_scores = [bpr_model.score(uidx)[iidx] for uidx, iidx in list(X_test)]
# wmf_scores = [wmf_model.score(uidx)[iidx] for uidx, iidx in X_test]

# df = pd.DataFrame({'user': test_set.uir_tuple[0], 'item': test_set.uir_tuple[1], 'groundtruth rating': test_set.uir_tuple[2], 'bpr_rating': bpr_scores, 'wmf_rating': wmf_scores})
# df.head()


### Interpreting Results

In [4]:
# Download some information of MovieLens 100K dataset
item_df = pd.read_csv(
  cache("http://files.grouplens.org/datasets/movielens/ml-100k/u.item"), 
  sep="|", encoding="ISO-8859-1",
  names=["ItemID", "Title", "Release Date", "Video Release Date", "IMDb URL", 
         "unknown", "Action", "Adventure", "Animation", "Children's", "Comedy", 
         "Crime", "Documentary", "Drama", "Fantasy", "Film-Noir", "Horror", 
         "Musical", "Mystery", "Romance", "Sci-Fi", "Thriller", "War", "Western"]
).set_index("ItemID").drop(columns=["Video Release Date", "IMDb URL", "unknown"])
     

In [11]:
from IPython.display import display
UIDX = 3
TOPK = 10

item_idx2id = list(train_set.item_ids)

# take a step back and take a look at training data
# make sense of the data and recommendations

training_data_df = pd.DataFrame(zip(*train_set.uir_tuple))
training_data_df.columns = ['user_idx', 'item_idx', 'rating']
training_data_df['item_id'] = training_data_df.apply(lambda row: item_idx2id[int(row['item_idx'])], axis=1)

filter_df = training_data_df[(training_data_df['rating'] == 5.0) & (training_data_df['user_idx'] == UIDX)]
filter_df = item_df.loc[[int(item_id) for item_id in filter_df["item_id"]]]
filter_df = filter_df.select_dtypes(np.number).sum() # sum number of movies in training set by movie genre

# sum of genres in training data
print("Movies rated 5.0 by user index 3 in training data")
display(filter_df.sort_values(ascending=False))

Movies rated 5.0 by user index 3 in training data


Drama          71
Comedy         39
Romance        32
Action         30
Thriller       29
Adventure      19
War            15
Crime          12
Sci-Fi          9
Mystery         8
Children's      5
Horror          5
Musical         4
Film-Noir       2
Animation       2
Western         2
Documentary     1
Fantasy         0
dtype: int64

As shown above in the training data, the top genres for user index 3 with movies rated 5.0 include 'Drama', 'Comedy', 'Romance', 'Action' and 'Thriller'.

Let's take a look at what movies and its coresponding genres are recommended to this user by the BPR and WMF models respectively.

In [7]:

bpr_recommendations, bpr_scores = bpr_model.rank(UIDX)
wmf_recommendations, wmf_scores = wmf_model.rank(UIDX)

# Top K recommended items for each model
bpr_topk = [item_idx2id[iidx] for iidx in bpr_recommendations[:TOPK]]
wmf_topk = [item_idx2id[iidx] for iidx in wmf_recommendations[:TOPK]]

print("BPR Top K Item IDs:", bpr_topk)
print("WMF Top K Item IDs:", wmf_topk)

print("\nTop K BPR Item recommendations with Movie Genre")
display(item_df.loc[[int(iid) for iid in bpr_topk]])

print("\nTop K WMF Item recommendations with Movie Genre")
display(item_df.loc[[int(iid) for iid in wmf_topk]])

BPR Top K Item IDs: ['402', '313', '88', '739', '69', '215', '781', '300', '82', '258']
WMF Top K Item IDs: ['318', '272', '64', '313', '315', '191', '268', '196', '127', '87']

Top K BPR Item recommendations with Movie Genre


,Title,Release Date,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
ItemID,,,,,,,,,,,,,,,,,,,,
402,Ghost (1990),01-Jan-1990,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0
313,Titanic (1997),01-Jan-1997,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0
88,Sleepless in Seattle (1993),01-Jan-1993,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
739,Pretty Woman (1990),01-Jan-1990,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
69,Forrest Gump (1994),01-Jan-1994,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0
215,Field of Dreams (1989),01-Jan-1989,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
781,French Kiss (1995),01-Jan-1995,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
300,Air Force One (1997),01-Jan-1997,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
82,Jurassic Park (1993),01-Jan-1993,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0



Top K WMF Item recommendations with Movie Genre


,Title,Release Date,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
ItemID,,,,,,,,,,,,,,,,,,,,
318,Schindler's List (1993),01-Jan-1993,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0
272,Good Will Hunting (1997),01-Jan-1997,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
64,"Shawshank Redemption, The (1994)",01-Jan-1994,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
313,Titanic (1997),01-Jan-1997,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0
315,Apt Pupil (1998),23-Oct-1998,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0
191,Amadeus (1984),01-Jan-1984,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0
268,Chasing Amy (1997),01-Jan-1997,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0
196,Dead Poets Society (1989),01-Jan-1989,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
127,"Godfather, The (1972)",01-Jan-1972,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0


It seems that the BPR model tend to recommend the 'Comedy' and 'Romance' genres, while the WMF model tend to recommend mainly the 'Drama' genre.

Let's see what happens when we ensemble these two models. 

## 2. Simple model ensembling by Borda Count

We will ensemble the two models using the Borda Count method. The Borda Count method is a simple voting method that ranks the items based on the sum of their ranks from each model.

Assuming that we have a list of 5 items, the Borda Count method works as follows:

1. For each model, rank the items from 1 to 5 based on the predicted scores.
2. Sum the ranks of each item across all models.
3. Sort the items based on the sum of their ranks.
4. The top-ranked item is the final recommendation.
5. Repeat the process for the next user.

Given the below example for a random user 123:

| Rank | Model 1 | Model 2 | Model 3 | Allocated Points (N - rank) |
|------|---------|---------|---------|-----------------------------|
| 1    | A       | D       | E       | 5 - 1 = 4                   |
| 2    | B       | C       | A       | 5 - 2 = 3                   |
| 3    | C       | A       | B       | 5 - 3 = 2                   |
| 4    | D       | B       | D       | 5 - 4 = 1                   |
| 5    | E       | E       | C       | 5 - 5 = 0                   |

The final ranking is as follows after summing them all up

| Item | Total Points     |
|------|------------------|
| A    | 4 + 2 + 3 = 9    |
| B    | 3 + 1 + 2 = 6    |
| D    | 1 + 4 + 1 = 6    |
| C    | 2 + 3 + 0 = 5    |
| E    | 0 + 0 + 4 = 4    |

New ranking: A > B, D > C > E


Lets implement this method below by using pandas `DataFrame` for data manipulation.

In [9]:
# todo: group by users

rank_df = pd.DataFrame({
    "ItemID": item_idx2id,
    "BPR Score": bpr_scores,
    "WMF Score": wmf_scores
})

# Obtain ranks of the items based on the scores
rank_df["BPR Rank"] = rank_df["BPR Score"].rank(ascending=False)
rank_df["WMF Rank"] = rank_df["WMF Score"].rank(ascending=False)

total_items = len(rank_df) # 1651 items

# Get Borda Points for each of the models. Borda points are calculated as (total items - rank)
rank_df["BPR Borda Points"] = total_items - rank_df["BPR Rank"]
rank_df["WMF Borda Points"] = total_items - rank_df["WMF Rank"]

rank_df["Total Points"] = rank_df["BPR Borda Points"] + rank_df["WMF Borda Points"]

display(rank_df)

,ItemID,BPR Score,WMF Score,BPR Rank,WMF Rank,BPR Borda Points,WMF Borda Points,Total Points
0,381,2.276938,3.636231,207.0,249.0,1444.0,1402.0,2846.0
1,602,-1.072662,1.992260,1076.0,739.0,575.0,912.0,1487.0
2,431,1.191264,3.058641,439.0,404.0,1212.0,1247.0,2459.0
3,875,0.757081,2.234277,555.0,652.0,1096.0,999.0,2095.0
4,182,1.652270,4.467779,341.0,88.0,1310.0,1563.0,2873.0
...,...,...,...,...,...,...,...,...
1646,1635,-2.083377,0.554031,1389.0,1294.0,262.0,357.0,619.0
1647,1650,-2.147377,0.737527,1423.0,1193.0,228.0,458.0,686.0
1648,1647,-1.982187,0.554163,1342.0,1293.0,309.0,358.0,667.0
1649,1663,-2.288501,0.338493,1477.0,1391.0,174.0,260.0,434.0


Now that we have a joint score, let's rerank this list and to provide the ensembled model's recommendation.

In [10]:
reranked_df = rank_df.sort_values("Total Points", ascending=False)

print("Re-ranked Top K Item recommendations")
display(reranked_df)

borda_count_topk = reranked_df["ItemID"].values[:TOPK]
print("\nTop K Ensembled Item recommendations")
print(borda_count_topk)

print("\nBorda Count recommendations with Movie Genre")
item_df.loc[[int(i) for i in borda_count_topk]]

Re-ranked Top K Item recommendations


,ItemID,BPR Score,WMF Score,BPR Rank,WMF Rank,BPR Borda Points,WMF Borda Points,Total Points
152,313,4.810898,5.686780,2.0,4.0,1649.0,1647.0,3296.0
126,69,4.345708,5.133685,5.0,16.0,1646.0,1635.0,3281.0
604,215,4.311501,5.128251,6.0,17.0,1645.0,1634.0,3279.0
308,216,4.014839,5.196355,16.0,12.0,1635.0,1639.0,3274.0
382,655,3.999588,5.172272,17.0,13.0,1634.0,1638.0,3272.0
...,...,...,...,...,...,...,...,...
578,1122,-2.827056,-0.351094,1616.0,1641.0,35.0,10.0,45.0
1507,1332,-2.926263,-0.188923,1631.0,1629.0,20.0,22.0,42.0
1639,1593,-2.918111,-0.334865,1630.0,1639.0,21.0,12.0,33.0
378,1597,-2.938648,-0.266447,1633.0,1637.0,18.0,14.0,32.0



Top K Ensembled Item recommendations
['313' '69' '215' '216' '655' '237' '300' '204' '64' '315']

Borda Count recommendations with Movie Genre


,Title,Release Date,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
ItemID,,,,,,,,,,,,,,,,,,,,
313,Titanic (1997),01-Jan-1997,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0
69,Forrest Gump (1994),01-Jan-1994,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0
215,Field of Dreams (1989),01-Jan-1989,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
216,When Harry Met Sally... (1989),01-Jan-1989,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
655,Stand by Me (1986),01-Jan-1986,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
237,Jerry Maguire (1996),13-Dec-1996,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0
300,Air Force One (1997),01-Jan-1997,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
204,Back to the Future (1985),01-Jan-1985,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0
64,"Shawshank Redemption, The (1994)",01-Jan-1994,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


Now, it seems that the ensembled model is able to provide a more diverse set of recommendations. Recommendations from movie genres such as 'Drama', 'Comedy', 'Romance', 'Action' and 'Thriller' are now included in the top 10 recommendations.

In the next section, we will see how we could further add more models to the ensemble.

## 3. Adding more models to the Borda Count ensemble

We can easily add more models to the ensemble by training them and adding them. One approach is to train a model with different initializations using different random seeds. By adding multiple similar models of different random seeds (`seed=123`), some models could perform better for a set of users, while other models could perform better for another set of users.

By ensembling these models, we could potentially achieve better performance when combined.

Let's try adding a few more similar models with different random seed initializations.

In [12]:
# BPR models with different seeds
# bpr_seed_123 = BPR(name="BPR_123", k=10, max_iter=100, learning_rate=0.01, lambda_reg=0.001, seed=123)
# bpr_seed_456 = BPR(name="BPR_456", k=10, max_iter=100, learning_rate=0.01, lambda_reg=0.001, seed=456)
# bpr_seed_789 = BPR(name="BPR_789", k=10, max_iter=100, learning_rate=0.01, lambda_reg=0.001, seed=789)
# bpr_seed_888 = BPR(name="BPR_888", k=10, max_iter=100, learning_rate=0.01, lambda_reg=0.001, seed=888)
# bpr_seed_999 = BPR(name="BPR_999", k=10, max_iter=100, learning_rate=0.01, lambda_reg=0.001, seed=999)

# WMF models with different seeds
wmf_model_123 = WMF(name="WMF_123", k=10, max_iter=300, a=1.0, b=0.1, learning_rate=0.001, lambda_u=0.01, lambda_v=0.01, seed=123)
wmf_model_456 = WMF(name="WMF_456", k=10, max_iter=300, a=1.0, b=0.1, learning_rate=0.001, lambda_u=0.01, lambda_v=0.01, seed=456)
wmf_model_789 = WMF(name="WMF_789", k=10, max_iter=300, a=1.0, b=0.1, learning_rate=0.001, lambda_u=0.01, lambda_v=0.01, seed=789)
wmf_model_888 = WMF(name="WMF_888", k=10, max_iter=300, a=1.0, b=0.1, learning_rate=0.001, lambda_u=0.01, lambda_v=0.01, seed=888)
wmf_model_999 = WMF(name="WMF_999", k=10, max_iter=300, a=1.0, b=0.1, learning_rate=0.001, lambda_u=0.01, lambda_v=0.01, seed=999)

models = [wmf_model_123, wmf_model_456, wmf_model_789, wmf_model_888, wmf_model_999]

experiment = Experiment(rs, models, metrics, user_based=True).run()


[WMF_123] Training started!


100%|██████████| 300/300 [00:16<00:00, 18.58it/s, loss=173]


Learning completed!

[WMF_123] Evaluation started!


Ranking: 100%|██████████| 940/940 [00:00<00:00, 2199.21it/s]



[WMF_456] Training started!


...


[WMF_999] Training started!


100%|██████████| 300/300 [00:17<00:00, 17.60it/s, loss=171]


Learning completed!

[WMF_999] Evaluation started!


Ranking: 100%|██████████| 940/940 [00:00<00:00, 1747.96it/s]



TEST:
...
        | Precision@10 | Recall@10 | Train (s) | Test (s)
------- + ------------ + --------- + --------- + --------
WMF_123 |       0.3131 |    0.2081 |   16.3334 |   0.4317
WMF_456 |       0.3033 |    0.1997 |   17.3587 |   0.4754
WMF_789 |       0.3094 |    0.2014 |   18.4598 |   0.7776
WMF_888 |       0.3106 |    0.2041 |   18.6320 |   0.4741
WMF_999 |       0.3119 |    0.2035 |   17.1849 |   0.5427



Based on the results, we can see that even within the same model, the results can vary. 

Let's try ensembling all these models together into 1 single model by Borda Count, and look at its recommendations.

In [18]:
rank_2_df = pd.DataFrame({
    "ItemID": item_idx2id,
})

rank_2_df["Total Points"] = 0

for model in models:
    name = model.name
    recommendations, scores = model.rank(UIDX)
    rank_2_df[name + "_rating"] = scores
    rank_2_df[name + "_rank"] = rank_2_df[name + "_rating"].rank(ascending=False)
    rank_2_df[name + "_points"] = total_items - rank_2_df[name + "_rank"]
    rank_2_df["Total Points"] = rank_2_df["Total Points"] + rank_2_df[name + "_points"]
    
print("Model score calculation:")
display(rank_2_df.head(10))

print("\nRe-ranked Top K Item recommendations")
borda_count_2_topk = list(rank_2_df.sort_values("Total Points", ascending=False)["ItemID"].values[:TOPK])
print(borda_count_2_topk)

print("\nBorda Count recommendations with Movie Genre")
display(item_df.loc[[int(i) for i in borda_count_2_topk]])

Model score calculation:


,ItemID,Total Points,WMF_123_rating,WMF_123_rank,WMF_123_points,WMF_456_rating,WMF_456_rank,WMF_456_points,WMF_789_rating,WMF_789_rank,WMF_789_points,WMF_888_rating,WMF_888_rank,WMF_888_points,WMF_999_rating,WMF_999_rank,WMF_999_points
0,381,7484.0,4.442100,83.0,1568.0,4.361825,94.0,1557.0,3.918090,179.0,1472.0,4.319282,106.0,1545.0,3.446604,309.0,1342.0
1,602,4774.0,1.888972,746.0,905.0,2.309264,637.0,1014.0,1.898686,752.0,899.0,1.428972,908.0,743.0,2.973416,438.0,1213.0
2,431,6408.0,2.890685,455.0,1196.0,3.622964,265.0,1386.0,3.075909,384.0,1267.0,3.420203,310.0,1341.0,3.006708,433.0,1218.0
3,875,4683.0,2.766791,487.0,1164.0,1.201590,1014.0,637.0,2.177860,661.0,990.0,2.048416,696.0,955.0,2.071572,714.0,937.0
4,182,6783.0,3.926973,194.0,1457.0,3.566039,286.0,1365.0,3.245656,347.0,1304.0,3.136099,383.0,1268.0,3.617298,262.0,1389.0
5,1074,6951.0,3.938466,189.0,1462.0,3.212995,378.0,1273.0,3.850867,192.0,1459.0,4.041566,156.0,1495.0,3.184880,389.0,1262.0
6,286,7985.0,4.621738,57.0,1594.0,4.241239,118.0,1533.0,4.767173,46.0,1605.0,4.959227,26.0,1625.0,4.834836,23.0,1628.0
7,496,7892.0,4.792030,36.0,1615.0,4.249075,115.0,1536.0,4.390133,97.0,1554.0,4.488886,77.0,1574.0,4.710654,38.0,1613.0
8,15,8091.0,5.207282,6.0,1645.0,4.720450,28.0,1623.0,4.525326,71.0,1580.0,4.736200,48.0,1603.0,5.070928,11.0,1640.0
9,184,4828.0,1.912682,740.0,911.0,2.203307,676.0,975.0,2.548484,532.0,1119.0,2.134595,667.0,984.0,1.750076,812.0,839.0



Re-ranked Top K Item recommendations
['313', '272', '50', '64', '191', '318', '181', '402', '66', '732']

Borda Count recommendations with Movie Genre


,Title,Release Date,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
ItemID,,,,,,,,,,,,,,,,,,,,
313,Titanic (1997),01-Jan-1997,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0
272,Good Will Hunting (1997),01-Jan-1997,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
50,Star Wars (1977),01-Jan-1977,1,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0
64,"Shawshank Redemption, The (1994)",01-Jan-1994,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
191,Amadeus (1984),01-Jan-1984,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0
318,Schindler's List (1993),01-Jan-1993,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0
181,Return of the Jedi (1983),14-Mar-1997,1,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0
402,Ghost (1990),01-Jan-1990,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0
66,While You Were Sleeping (1995),01-Jan-1995,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0


## 4. Advanced Model Ensembling

We could continue by thinking of this as a meta-learning problem. We could treat recommendations of each base model as features and train a meta-learner to predict the final recommendation.

This could be any ML model such as a Linear Regression, Random Forest, Gradient Boosting, or even a Neural Network.


In this example, we will use a simple Linear Regression model to predict the final recommendation.

In [47]:
# We continue using UIDX = 0 and TOPK = 10
rank_3_df = pd.DataFrame({
    "ItemID": item_idx2id,
})

print(len(item_idx2id))
print(train_set.uir_tuple[0])
print(train_set.uir_tuple[1])

# Lets get all the scores for the models trained in Part 3.
training_df = pd.DataFrame(zip(*train_set.uir_tuple))
training_df.columns = ['user_idx', 'item_idx', 'ground_rating']
test_df = pd.DataFrame(zip(*test_set.uir_tuple))
test_df.columns = ['user_idx', 'item_idx', 'ground_rating']
test_df['item_id'] = test_df.apply(lambda row: item_idx2id[int(row['item_idx'])], axis=1)

for model in models:
    name = model.name
    training_df[name + "_rating"] = training_df.apply(lambda row: model.score(int(row['user_idx']), int(row['item_idx'])), axis=1)
    test_df[name + "_rating"] = test_df.apply(lambda row: model.score(int(row['user_idx']), int(row['item_idx'])), axis=1)

display(training_df.tail(10))

X_train = training_df[['WMF_123_rating', 'WMF_456_rating', 'WMF_789_rating', 'WMF_888_rating', 'WMF_999_rating']]
y_train = training_df['ground_rating']
X_test = test_df[['WMF_123_rating', 'WMF_456_rating', 'WMF_789_rating', 'WMF_888_rating', 'WMF_999_rating']]

from sklearn import linear_model

regr = linear_model.LinearRegression()
regr.fit(X_train, y_train)

y_pred = regr.predict(X_test)

test_df["Linear Regression Rating"] = y_pred

# sort by predicted rating
UIDX = 3
TOPK = 10

test_df = test_df.sort_values("Linear Regression Rating", ascending=False)

top_item_ids = test_df[test_df['user_idx'] == UIDX]['item_id'].values[:TOPK]
print("Top K Item recommendations")
print(top_item_ids)

print("\nTop K Item recommendations with Movie Genre")
display(item_df.loc[[int(i) for i in top_item_ids]])


1651
[  0   1   2 ... 281 262  14]
[  0   1   2 ... 296 533 552]


,user_idx,item_idx,ground_rating,WMF_123_rating,WMF_456_rating,WMF_789_rating,WMF_888_rating,WMF_999_rating
79990,64,720,2.0,2.105613,1.988491,1.815853,2.098003,1.967414
79991,75,19,2.0,2.399415,1.809463,2.316859,3.028243,2.509972
79992,815,169,3.0,1.310617,1.288808,0.964995,1.215850,1.164233
79993,427,690,5.0,3.048433,3.200139,3.178525,3.111808,3.123214
79994,508,232,3.0,2.845090,3.041863,3.040719,2.961675,2.780494
79995,407,642,4.0,3.798587,3.373164,3.742901,4.002794,3.773691
79996,614,1018,3.0,2.806572,2.718426,2.973318,2.739293,2.827957
79997,281,296,5.0,3.882876,4.060247,3.801868,4.152957,3.775539
79998,262,533,4.0,2.641438,3.394031,2.990592,2.730569,2.537793
79999,14,552,3.0,2.757534,2.722447,2.874388,2.416764,2.739604


Top K Item recommendations
['272' '66' '588' '258' '125' '111' '97' '742' '451' '127']

Top K Item recommendations with Movie Genre


,Title,Release Date,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
ItemID,,,,,,,,,,,,,,,,,,,,
272,Good Will Hunting (1997),01-Jan-1997,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
66,While You Were Sleeping (1995),01-Jan-1995,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
588,Beauty and the Beast (1991),01-Jan-1991,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0
258,Contact (1997),11-Jul-1997,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0
125,Phenomenon (1996),29-Jun-1996,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0
111,"Truth About Cats & Dogs, The (1996)",26-Apr-1996,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
97,Dances with Wolves (1990),01-Jan-1990,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1
742,Ransom (1996),08-Nov-1996,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0
451,Grease (1978),01-Jan-1978,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0


To reiterate, the top genres for user index 3 with movies rated 5.0 include 'Drama', 'Comedy', 'Romance', 'Action' and 'Thriller'.

### Applying Random Forest model



In [48]:
randomforest_model = RandomForestRegressor(n_estimators=200, random_state=42) # sklearn Random Forest model
randomforest_model.fit(X_train, y_train)

y_pred = randomforest_model.predict(X_test)

UIDX = 3
TOPK = 10

test_df["Random Forest Rating"] = y_pred

# Sort the items based on the score
test_df = test_df.sort_values("Random Forest Rating", ascending=False)

top_item_ids = test_df[test_df['user_idx'] == UIDX]['item_id'].values[:TOPK]
print("Top K Item recommendations")
print(top_item_ids)

print("\nTop K Item recommendations with Movie Genre")
display(item_df.loc[[int(i) for i in top_item_ids]])

Top K Item recommendations
['742' '106' '405' '307' '164' '763' '1300' '942' '66' '127']

Top K Item recommendations with Movie Genre


,Title,Release Date,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
ItemID,,,,,,,,,,,,,,,,,,,,
742,Ransom (1996),08-Nov-1996,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0
106,Diabolique (1996),01-Jan-1996,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0
405,Mission: Impossible (1996),22-May-1996,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
307,"Devil's Advocate, The (1997)",01-Jan-1997,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,1,0,0
164,"Abyss, The (1989)",01-Jan-1989,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0
763,Happy Gilmore (1996),16-Feb-1996,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1300,'Til There Was You (1997),30-May-1997,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0
942,What's Love Got to Do with It (1993),01-Jan-1993,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
66,While You Were Sleeping (1995),01-Jan-1995,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
